In [1]:
import pandas as pd
import json

In [2]:
def excel_sheet_to_json(file_path, sheet_name=None):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
        task_list = []

        for _, row in df.iterrows():
            task = {
                "task_id": str(row['task_id']),
                "task_description": row['task_desc'],
                "task_type": row['task_type'],
                "start_url": row['start_url'],
                "require_login": str(row['require_login']).strip().lower() == 'yes',  # or "true"
                "eval": {
                    "eval_type": [],
                }
            }

            # Parse từng loại match nếu có dữ liệu
            if pd.notna(row.get("string_match")):
                task["eval"]["eval_type"].append("string_match")
                try:
                    task["eval"]["string_match"] = json.loads(row["string_match"])
                except json.JSONDecodeError as e:
                    raise ValueError(f"Lỗi parse string_match: {e}")

            if pd.notna(row.get("regex_match")):
                task["eval"]["eval_type"].append("regex_match")
                try:
                    task["eval"]["regex_match"] = json.loads(row["regex_match"])
                except json.JSONDecodeError as e:
                    raise ValueError(f"Lỗi parse regex_match: {e}")

            if pd.notna(row.get("url_match")):
                task["eval"]["eval_type"].append("url_match")
                try:
                    task["eval"]["url_match"] = json.loads(row["url_match"])
                except json.JSONDecodeError as e:
                    raise ValueError(f"Lỗi parse url_match: {e}")

            if pd.notna(row.get("dom_match")):
                task["eval"]["eval_type"].append("dom_match")
                try:
                    task["eval"]["dom_match"] = json.loads(row["dom_match"])
                except json.JSONDecodeError as e:
                    raise ValueError(f"Lỗi parse dom_match: {e}")

            if pd.notna(row.get("multiset_match")):
                task["eval"]["eval_type"].append("multiset_match")
                try:
                    task["eval"]["multiset_match"] = json.loads(row["multiset_match"])
                except json.JSONDecodeError as e:
                    raise ValueError(f"Lỗi parse multiset_match: {e}")

            if pd.notna(row.get("list_match")):
                task["eval"]["eval_type"].append("list_match")
                try:
                    task["eval"]["list_match"] = json.loads(row["list_match"])
                except json.JSONDecodeError as e:
                    raise ValueError(f"Lỗi parse list_match: {e}")

            task_list.append(task)

        # return json.dumps(task_list, indent=4, ensure_ascii=False)
        return task_list
    except Exception as e:
        return f"Lỗi khi chuyển đổi: {e}"

In [3]:
results = excel_sheet_to_json('../data/App_Tasks.xlsx', 'test')
 # Lưu ra file JSON
with open("../data/tasks_temp.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)


In [4]:
tasks_json = json.dumps(results, indent=4, ensure_ascii=False)
print(tasks_json)
# Ghi ra file
# with open("tasks_json.json", "w", encoding="utf-8") as f:
#     f.write(tasks_json)

[
    {
        "task_id": "1",
        "task_description": "Retrieve the name of the first product listed in the \"Gear > Watches\" category",
        "task_type": "lookup",
        "start_url": "__SHOPPING__",
        "require_login": false,
        "eval": {
            "eval_type": [
                "string_match"
            ],
            "string_match": {
                "match_type": "exact",
                "match_value": "Aim Analog Watch"
            }
        }
    },
    {
        "task_id": "2",
        "task_description": "Rate the product “Hero Hoodie” with 4 stars.",
        "task_type": "operation",
        "start_url": "__SHOPPING__",
        "require_login": true,
        "eval": {
            "eval_type": [
                "string_match"
            ],
            "string_match": {
                "match_type": "semantic",
                "match_value": "Veronica Costello\n6146 Honey Bluff Parkway\nCalder, Michigan, 49628-7978\nUnited States\nT: (555) 229-3326"
   